In [ ]:
import face_recognition
import cv2
import numpy as np
import csv
from datetime import datetime

cap = cv2.VideoCapture(0)

img1 = face_recognition.load_image_file("person1.jpg")
img1_encoding = face_recognition.face_encodings(img1)[0]

img2 = face_recognition.load_image_file("person2.jpg")
img2_encoding = face_recognition.face_encodings(img2)[0]

known_face_encoding = [img1_encoding, img2_encoding]
known_faces_names = ["emp1", "emp2"]
students = known_faces_names.copy()

face_locations = []
face_encodings = []
face_names = []
s = True

now = datetime.now()
current_date = now.strftime("%Y-%m-%d")

f = open(current_date + '.csv', 'w+', newline='')
lnwriter = csv.writer(f)
column_names = ["Name", "Time"]
lnwriter.writerow(column_names)

while True:
    _, frame = cap.read()
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = np.ascontiguousarray(small_frame[:, :, ::-1])

    if s:
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
        face_names = []

        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(known_face_encoding, face_encoding)
            name = ""
            face_distance = face_recognition.face_distance(known_face_encoding, face_encoding)
            best_match_index = np.argmin(face_distance)

            if matches[best_match_index]:
                name = known_faces_names[best_match_index]

            face_names.append(name)

            if name in known_faces_names:
                if name in students:
                    students.remove(name)
                    current_time = now.strftime("%H-%M-%S")
                    lnwriter.writerow([name, current_time])

    cv2.imshow("attendance", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
f.close()
